https://www.kaggle.com/competitions/reconocimiento-de-expresiones-faciales

In [29]:
import numpy as np
import pandas as pd
import os
import cv2
from skimage.io import imread
from sklearn.model_selection import train_test_split

from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

import tensorflow as tf
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.models import Model
from tensorflow.keras import layers, models
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam


In [2]:
def read_data(directorio):
    pixel = [] 
    expression = []
    for folder in os.listdir(directorio):
        print(folder)
        if os.path.isdir('/'.join([directorio, folder])):
            for file in os.listdir('/'.join([directorio, folder])):

                image = imread('/'.join([directorio, folder, file]))

                pixel.append(image)
                expression.append(folder)

    return np.array(pixel),np.array(expression)

def read_data_test(directorio):
    pixel = [] 
    for folder in os.listdir(directorio):
        #print(folder)
        #if os.path.isdir('/'.join([directorio, folder])):
        #for file in os.listdir('/'.join([directorio, folder])):

        image = imread('/'.join([directorio, folder]))

        pixel.append(image)


    return np.array(pixel)

In [3]:
X_train_array, y_train_array = read_data('./data/train')

true_test = read_data_test('./data/test')

# Barajo las dos arrays en el mismo orden:

indices = np.random.permutation(X_train_array.shape[0])
X_train_shuffled = X_train_array[indices]
y_train_shuffled = y_train_array[indices]

# ---

X_test = X_train_shuffled[:4611]
X_val = X_train_shuffled[4612:9222]
X_train = X_train_shuffled[9223:]
y_test = y_train_shuffled[:4611]
y_val = y_train_shuffled[4612:9222]
y_train = y_train_shuffled[9223:]

angry
disgust
fear
happy
neutral
sad
surprise


In [4]:
################# MODIFICACIONES ###########################

In [5]:
X_train = X_train / 255
X_val = X_val / 255
X_test = X_test / 255

In [6]:
train_rgb_images = np.stack((X_train,) * 3, axis=-1)
val_rgb_images = np.stack((X_val,) * 3, axis=-1)
test_rgb_images = np.stack((X_test,) * 3, axis=-1)

In [7]:
##########################################################

In [8]:
dict_emotions = {
    'angry':0,
    'disgust':1,
    'fear':2,
    'happy':3,
    'neutral':4,
    'sad':5,
    'surprise':6
}

In [9]:
y_train_mapped = np.array([dict_emotions[emotion] for emotion in y_train])
y_val_mapped = np.array([dict_emotions[emotion] for emotion in y_val])
y_test_mapped = np.array([dict_emotions[emotion] for emotion in y_test])

In [10]:
from tensorflow.keras.utils import to_categorical

train_labels_one_hot = to_categorical(y_train_mapped, num_classes=7)
val_labels_one_hot = to_categorical(y_val_mapped, num_classes=7)
test_labels_one_hot = to_categorical(y_test_mapped, num_classes=7)

# MODELO

In [22]:
base_model = MobileNet(weights='imagenet', include_top=False, input_shape=(48, 48, 3))

C:\Users\annav\AppData\Local\Temp\ipykernel_7604\989208286.py:1: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  base_model = MobileNet(weights='imagenet', include_top=False, input_shape=(48, 48, 3))


In [23]:
for layer in base_model.layers:
    layer.trainable = False

In [26]:
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(48, 48, 3)))  # Asegúrate de que el input_shape tenga 3 canales
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(7, activation='softmax'))

c:\Users\annav\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [27]:
model_base_output = base_model.output
x = layers.Flatten()(model_base_output)
x = layers.Dense(512, activation='relu')(x)
output = layers.Dense(7, activation='softmax')(x)

In [30]:
model = tf.keras.models.Model(inputs=base_model.input, outputs=output)

In [31]:
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

In [15]:
y_train_mapped.shape

(19598,)

In [16]:
train_rgb_images.shape

(19598, 48, 48, 3)

In [32]:
model.fit(train_rgb_images, train_labels_one_hot, epochs=10, batch_size=32, validation_data=(val_rgb_images, val_labels_one_hot))

Epoch 1/10
613/613 ━━━━━━━━━━━━━━━━━━━━ 33s 50ms/step - accuracy: 0.2716 - loss: 1.7701 - val_accuracy: 0.2928 - val_loss: 1.7469
Epoch 2/10
613/613 ━━━━━━━━━━━━━━━━━━━━ 27s 44ms/step - accuracy: 0.2950 - loss: 1.7255 - val_accuracy: 0.2911 - val_loss: 1.7404
Epoch 3/10
613/613 ━━━━━━━━━━━━━━━━━━━━ 25s 41ms/step - accuracy: 0.3019 - loss: 1.7185 - val_accuracy: 0.2980 - val_loss: 1.7361
Epoch 4/10
613/613 ━━━━━━━━━━━━━━━━━━━━ 24s 39ms/step - accuracy: 0.3105 - loss: 1.7067 - val_accuracy: 0.2948 - val_loss: 1.7317
Epoch 5/10
613/613 ━━━━━━━━━━━━━━━━━━━━ 30s 48ms/step - accuracy: 0.3109 - loss: 1.7049 - val_accuracy: 0.2805 - val_loss: 1.7453
Epoch 6/10
613/613 ━━━━━━━━━━━━━━━━━━━━ 30s 50ms/step - accuracy: 0.3102 - loss: 1.6986 - val_accuracy: 0.2837 - val_loss: 1.7338
Epoch 7/10
613/613 ━━━━━━━━━━━━━━━━━━━━ 27s 44ms/step - accuracy: 0.3131 - loss: 1.6940 - val_accuracy: 0.2939 - val_loss: 1.7411
Epoch 8/10
613/613 ━━━━━━━━━━━━━━━━━━━━ 26s 43ms/step - accuracy: 0.3187 - loss: 1.6843 - 

In [33]:
model.evaluate(test_rgb_images,test_labels_one_hot)

145/145 ━━━━━━━━━━━━━━━━━━━━ 3s 22ms/step - accuracy: 0.3028 - loss: 1.7283


[1.7338836193084717, 0.29819995164871216]

In [34]:
predictions = model.predict(test_rgb_images)

145/145 ━━━━━━━━━━━━━━━━━━━━ 10s 46ms/step


In [35]:
predicted_classes = np.argmax(predictions, axis=1)

In [36]:
print(classification_report(y_test_mapped, predicted_classes, target_names=dict_emotions.keys()))

              precision    recall  f1-score   support

       angry       0.22      0.08      0.11       617
     disgust       1.00      0.01      0.03        77
        fear       0.22      0.09      0.13       690
       happy       0.32      0.65      0.43      1106
     neutral       0.27      0.15      0.19       771
         sad       0.25      0.32      0.28       793
    surprise       0.40      0.31      0.35       557

    accuracy                           0.30      4611
   macro avg       0.38      0.23      0.22      4611
weighted avg       0.29      0.30      0.26      4611



In [13]:
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation='relu')(x)
x = Dense(256, activation='relu')(x)
predictions = Dense(7, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)